In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
import tensorflow as tf
import os
import shutil
from zipfile import ZipFile

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv
/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip
/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip


In [2]:
path = "/kaggle/input/dogs-vs-cats-redux-kernels-edition/"

with ZipFile(path + "train.zip") as train_zip:
    train_zip.extractall("/kaggle/working")
    
with ZipFile(path + "test.zip") as test_zip:
    test_zip.extractall("/kaggle/working")

In [3]:
train_path = "/kaggle/working/train/"
test_path = "/kaggle/working/test/"

os.mkdir(train_path + "cat")
os.mkdir(train_path + "dog")

for file in os.listdir(train_path):
    if file.startswith("cat"):
        shutil.move(train_path + file, train_path + "cat")
    elif file.startswith("dog"):
        shutil.move(train_path + file, train_path + "dog")
    else:
        print(file + "is not cat or dog.")

In [4]:
seed = 0
IMG_SHAPE = (500, 500)

train_dataset = keras.preprocessing.image_dataset_from_directory(train_path,
                                                                 shuffle=True,
                                                                 subset="training",
                                                                 validation_split=0.1,
                                                                 image_size=IMG_SHAPE,
                                                                 seed=seed,
                                                                 batch_size=64)

valid_dataset = keras.preprocessing.image_dataset_from_directory(train_path,
                                                                shuffle=True,
                                                                subset="validation",
                                                                validation_split=0.1,
                                                                seed=seed,
                                                                image_size=IMG_SHAPE,
                                                                batch_size=64)

test_dataset = keras.preprocessing.image_dataset_from_directory(test_path,
                                                                shuffle=False,
                                                                label_mode=None,
                                                                image_size=IMG_SHAPE,
                                                                batch_size=64)

Found 25000 files belonging to 2 classes.
Using 22500 files for training.
Found 25000 files belonging to 2 classes.
Using 2500 files for validation.
Found 12500 files belonging to 1 classes.


In [5]:
augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.2)
])

In [6]:
base_model = keras.applications.InceptionResNetV2(include_top=False, weights="imagenet", input_shape=(IMG_SHAPE) + (3, ))

preprocess_input = keras.applications.inception_resnet_v2.preprocess_input

base_model.trainable = False

inputs = keras.layers.Input(shape=(IMG_SHAPE) + (3, ))

x = augmentation(inputs)

x = preprocess_input(inputs)

x = base_model(x, training=False)

x = keras.layers.GlobalAveragePooling2D()(x)

x = keras.layers.Dropout(0.5)(x)

outputs = keras.layers.Dense(1)(x)

model = keras.Model(inputs, outputs)

model.summary()

219055592/219055592 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 500, 500, 3)]     0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 500, 500, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 500, 500, 3)      0         
 a)                                                              
                                                                 
 inception_resnet_v2 (Functi  (None, 14, 14, 1536)     54336736  
 onal)                                                           
                                                                 
 global_average_pooling2d (G  (None, 1536)             0    

In [7]:
init_epoch = 1

optimizer = keras.optimizers.Adam()
loss = keras.losses.BinaryCrossentropy(from_logits=True)

model.compile(optimizer=optimizer,
              loss=loss,
              metrics=["accuracy"])

model.fit(train_dataset, epochs=init_epoch, validation_data=valid_dataset)

352/352 [==============================] - 400s 1s/step - loss: 0.1263 - accuracy: 0.9616 - val_loss: 0.0423 - val_accuracy: 0.9944


In [8]:
preds = tf.sigmoid(model.predict(test_dataset)).numpy().flatten()

196/196 [==============================] - 185s 928ms/step


In [9]:
preds_clipped = np.clip(preds, 0.01, 0.99)

In [10]:
dic = dict(zip([int(i[21:-4]) for i in test_dataset.file_paths], preds_clipped))
dic = dict(sorted(dic.items()))

In [11]:
df = pd.DataFrame.from_dict(dic, orient='index', columns=['label'])
df.index.name = 'id'
df.head()

,label
id,
1,0.987697
2,0.972984
3,0.895551
4,0.987241
5,0.010000


In [12]:
df.to_csv("submission.csv")